In [1]:
import numpy as np
from pathlib import Path
from read_raw_data import *

In [16]:
def users_month_Jan_to_March():
    months = set([
         "202001",
         "202002",
         "202003",
      ])
    file_names = sorted(Path("raw_data").rglob("*.txt"), reverse=True)
    set_users = set()
    with open("/home/alex/kayzhou/US_election/data/users_info_March_April.lj","w") as _file:
        for in_name in file_names:
            word = in_name.stem.split("-")[-1]
            if word in demo_files and in_name.parts[1] in months:
                print(in_name)
                for line in open(in_name):
                    try:
                        d = json.loads(line.strip())
                    except Exception:
                        print("JSON loading error")
                    if d["user"]["id"] in set_users:
                        continue
                    set_users.add(d["user"]["id"])
                    loc,profile_image_url=None,None
                    if 'location' in d['user']:
                        loc=d["user"]["location"]
                    if "profile_image_url" in d["user"]:
                        profile_image_url=d["user"]["profile_image_url"]
                    user_info={"user_id":d["user"]["id"],
                              "screen_name":d["user"]["screen_name"],
                              "location": loc,
                              "profile_image_url":profile_image_url}
                    _file.write(json.dumps(user_info, ensure_ascii=False) + "\n")


In [82]:
#read the file
users=set()
location=Counter()
c=0
with open('/home/alex/kayzhou/US_election/data/Final_users_list.lj','w') as _file:         
    for i in open("/home/alex/kayzhou/US_election/data/users_info_March_April.lj"):
        info=json.loads(i)
        if info['user_id'] not in users:
            users.add(info['user_id'])
            info['id'] =  info.pop('user_id')
            _file.write(json.dumps(info) + "\n")
    for i in open('/home/alex/kayzhou/US_election/data/all_users.lj'):
        if json.loads(json.loads(i))['id'] not in users:
            _file.write(json.dumps(json.loads(json.loads(i))) + "\n")
            users.add(json.loads(json.loads(i))['id'])  

            

In [90]:
loc_to_loc_1 = json.load(open("data/loc-to-loc.json"))
loc_to_loc_2 = json.load(open("data/loc-to-loc-20200604.json"))
loc_to_loc_3 = json.load(open("data/loc-to-loc-20200622.json"))
#merge all loc_to_loc in a unique dictionary
def DictListUpdate( lis1, lis2):
    for aLis1 in lis1:
        if aLis1 not in lis2:
            lis2.append(aLis1)
    return lis2
loc_to_loc= DictListUpdate( loc_to_loc_1,loc_to_loc_2)
loc_to_loc_final= DictListUpdate( loc_to_loc,loc_to_loc_3)
del loc_to_loc_1,loc_to_loc_2,loc_to_loc_3,loc_to_loc

In [91]:
loc_to_state_1 = json.load(open("data/loc-to-state.json"))
loc_to_state_2 = json.load(open("data/loc-to-state-20200604.json"))
loc_to_state_3 = json.load(open("data/loc-to-state-20200622.json"))
loc_to_state= DictListUpdate( loc_to_state_1,loc_to_state_2)
loc_to_state_final= DictListUpdate( loc_to_state,loc_to_state_3)
del loc_to_state_1,loc_to_state_2,loc_to_state_3,loc_to_state

In [36]:
missing=[]
#check those locations we already have
for i in location.most_common(15000):
    if i[0]:
        if i[0].lower() in loc_to_loc_final:
            continue
        else:
            missing.append(i)

In [40]:
print(missing)
#most of the missin are junks

[('Earth', 11843), ('Everywhere', 3413), ('Worldwide', 2472), (' ', 1758), ('Somewhere', 1736), ('SoCal', 1584), ('earth', 1537), ('Here', 1532), ('Hogwarts', 1048), ('Earth ', 1012), ('World', 949), ('everywhere', 904), ('somewhere', 889), ('HTX', 765), ('htx', 746), ('here', 733), ('Wakanda', 706), ('.', 654), ('ATX', 621), ('🇺🇸', 616), ('RVA', 612), ('Heaven', 607), ('🌎', 587), ('🇨🇦', 528), ('The Internet', 504), ('Everywhere ', 479), ('worldwide', 439), ('🌍', 435), ('🏳️\u200d🌈', 427), ('🇵🇭', 390), ('Twitter', 378), ('127.0.0.1', 321), ('Dreamville', 308), ('EARTH', 305), ('Somewhere ', 298), ('?', 296), ('SATX', 290), ('Online', 285), ('...', 281), ('¯\\_(ツ)_/¯', 266), ('DTX', 262), ('♡', 259), ('world', 255), ('ǝʌɐ uooɯʎǝuoɥ', 253), ('Anywhere', 252), ('idk', 249), ('atx', 230), ('heaven', 225), ('Astroworld', 219), ('\n', 217), ('🇮🇳', 215), ('dtx', 215), ('Mother Earth', 210), ('satx', 210), ('Reality', 209), ('Worldwide ', 208), ('🇳🇬', 207), ('somewhere ', 202), ('Hyrule', 201),

In [42]:
def check_from_loc(obj):
    try:
        complete_loc=loc_to_loc_final[obj]
    except:
        return False
    #loc_to_state
    try:
        State=loc_to_state_final[complete_loc.lower()]
        return State
    except:
        return False
def check_from_state(obj):
    try:
        State=loc_to_state_final[obj]
        return State
    except:
        return False

In [98]:
#Create unique file with users location:
ids=set()
c=0
with open('data/located_users_Jan_March.lj','w') as _file:
    for i in tqdm(users):
        if i['location']:
            if i['user_id'] not in ids:
                ids.add(i['user_id'])
                _st=check_from_loc(i['location'].lower())
                if _st:
                    c+=1
                    i['State']=_st
                    _file.write(json.dumps(i) + "\n")
                else:
                    _st=check_from_state(i['location'].lower())
                    if _st:
                        c+=1
                        i['State']=_st
                        _file.write(json.dumps(i) + "\n")
                    else:
                        continue


100%|██████████| 8491962/8491962 [00:21<00:00, 400719.37it/s]

In [103]:
collected_states=Counter()
for i in open('data/located_users_Jan_March.lj'):
    collected_states[json.loads(i)['State']]+=1
    

In [104]:
len(collected_states)
#ok 51 states. It make sense
#collected_states
#Swing_state

51

In [2]:
#swing state
SS=['AZ','CO','FL','GA','MI','MN','NV','NH','NC','OH','PA','WI']

In [3]:
already_analyzed=set()
for i in open('data/Users_swing_info_final.lj'):
    already_analyzed.add(json.loads(i)['id'])

In [4]:
swing_user=set()
with open('data/missing.lj',"w") as _file:
    for i in open('data/located_users_Jan_March.lj'):
        _temp=json.loads(i)
        if _temp['State'] in SS:
            if _temp['user_id'] not in already_analyzed:
                _file.write(json.dumps(_temp)+'\n')

In [37]:
c=[]
with open('data/Users_swing_info_final.lj','a') as _file:
    for i in open('data/Users_swing_missing.lj'):
        _file.write(json.dumps(i) + "\n")

In [312]:
c=[]
for i in  open('data/Users_swing_info_final.lj'):
    try:    
        c.append(json.loads(i)['id'])
    except:
        c.append(json.loads(json.loads(i))['user_id'])
print(len(c))

1052661


In [317]:
# Look at the city
#Find county
users=set()
loc_to_county = json.load(open("data/loc-to-county-20200622.json", encoding="utf8"))
users = []
for line in open("data/Final_users_list.lj"):
    d = json.loads(line.strip())
    uid = d["id"]
    if d['location']=='no_location':
        continue
    if d["location"]:
        loc = d["location"].lower().replace("\t", " ").replace("\n", " ")
        if loc in loc_to_county:
            w = loc_to_county[loc].split(", ")
            #if w[-1] != "United States" or len(w) < 2:
            #    continue
            # print(w)
            #county = w[-3] + ", " + w[-2]
            users.append({"id": uid, "loc": w[0]+','+w[1],'profile_image_url':d['profile_image_url']})

In [221]:
#check those alrady analyzed
count=0
c=set(c)
with open('data/county_users/County_users.lj', 'w') as outfile:
    for i in users:
        if i['id'] not in c:
            outfile.write(json.dumps(i)+'\n')

In [321]:
c=0
for i in open('data/county_users/County_users_analyzed.lj'):
    c+=1

In [322]:
c

34477

In [318]:
#check those alrady analyzed

swing=set(c)
to_save=[]
for i in users:
    if i['id'] in swing:
        to_save.append(i)

In [ ]:
final=[]
c=0
with open('data/county_users/County_users_swing.lj','w') as _file:
    for i in tqdm(to_save):
        c+=1
        _id=i['id']
        for j in analazyed:
            if isinstance(j, str):
                j=json.loads(j)
            if 'id' in j:
                if  j['id']==_id:
                    if j['faces'] and len(j['faces'])>1:
                        i['faces']=j['faces'][0]
                    else:
                        i['faces']=j['faces']
                    _file.write(json.dumps(i) + "\n")
                    break
            elif 'user_id' in j:
                if  j['user_id']==_id:
                    if j['faces'] and len(j['faces'])>1:
                        i['faces']=j['faces'][0]
                    else:
                        i['faces']=j['faces']
                    _file.write(json.dumps(i) + "\n")
                    break

                    
                
            






  0%|          | 0/724004 [00:00<?, ?it/s]




  0%|          | 2/724004 [00:00<25:28:29,  7.89it/s]




  0%|          | 5/724004 [00:00<21:18:55,  9.43it/s]




  0%|          | 13/724004 [00:00<16:36:09, 12.11it/s]




  0%|          | 19/724004 [00:00<12:58:29, 15.50it/s]




  0%|          | 23/724004 [00:00<10:50:51, 18.54it/s]




  0%|          | 26/724004 [00:01<10:34:08, 19.03it/s]




  0%|          | 29/724004 [00:01<14:13:35, 14.14it/s]




  0%|          | 32/724004 [00:01<16:17:28, 12.34it/s]




  0%|          | 34/724004 [00:02<21:05:59,  9.53it/s]




  0%|          | 37/724004 [00:02<20:31:04,  9.80it/s]




  0%|          | 41/724004 [00:02<18:37:53, 10.79it/s]




  0%|          | 43/724004 [00:03<25:19:49,  7.94it/s]




  0%|          | 45/724004 [00:03<32:07:28,  6.26it/s]




  0%|          | 47/724004 [00:03<28:59:13,  6.94it/s]




  0%|          | 50/724004 [00:03<25:41:06,  7.83it/s]




  0%|          | 51/724004 [00:04<32:49:18,  6.13it/s]




  0%| 

  0%|          | 379/724004 [00:26<13:11:10, 15.24it/s]




  0%|          | 381/724004 [00:27<18:35:25, 10.81it/s]




  0%|          | 383/724004 [00:27<23:33:48,  8.53it/s]




  0%|          | 386/724004 [00:27<21:20:16,  9.42it/s]




  0%|          | 389/724004 [00:28<22:43:22,  8.85it/s]




  0%|          | 391/724004 [00:28<24:27:39,  8.22it/s]




  0%|          | 394/724004 [00:28<24:42:13,  8.14it/s]




  0%|          | 397/724004 [00:29<25:37:42,  7.84it/s]




  0%|          | 398/724004 [00:29<36:21:48,  5.53it/s]




  0%|          | 405/724004 [00:29<28:14:37,  7.12it/s]




  0%|          | 407/724004 [00:30<33:32:24,  5.99it/s]




  0%|          | 408/724004 [00:30<31:39:27,  6.35it/s]




  0%|          | 411/724004 [00:30<25:27:54,  7.89it/s]




  0%|          | 413/724004 [00:30<22:04:24,  9.11it/s]




  0%|          | 415/724004 [00:31<20:52:15,  9.63it/s]




  0%|          | 418/724004 [00:31<18:30:03, 10.86it/s]




  0%|          | 420/724004 [00:31<17:52

  0%|          | 723/724004 [00:55<16:31:38, 12.16it/s]




  0%|          | 726/724004 [00:55<17:07:42, 11.73it/s]




  0%|          | 728/724004 [00:56<22:35:31,  8.89it/s]




  0%|          | 730/724004 [00:56<22:57:30,  8.75it/s]




  0%|          | 734/724004 [00:56<18:13:18, 11.03it/s]




  0%|          | 742/724004 [00:56<13:44:21, 14.62it/s]




  0%|          | 745/724004 [00:56<16:03:32, 12.51it/s]




  0%|          | 748/724004 [00:57<14:51:21, 13.52it/s]




  0%|          | 751/724004 [00:57<12:42:53, 15.80it/s]




  0%|          | 754/724004 [00:57<14:16:32, 14.07it/s]




  0%|          | 756/724004 [00:57<14:42:55, 13.65it/s]




  0%|          | 758/724004 [00:57<20:32:38,  9.78it/s]




  0%|          | 762/724004 [00:58<19:54:57, 10.09it/s]




  0%|          | 766/724004 [00:58<15:39:41, 12.83it/s]




  0%|          | 769/724004 [00:58<18:18:49, 10.97it/s]




  0%|          | 771/724004 [00:58<16:20:07, 12.30it/s]




  0%|          | 777/724004 [00:58<12:37

  0%|          | 1051/724004 [01:21<19:19:36, 10.39it/s]




  0%|          | 1053/724004 [01:21<16:34:14, 12.12it/s]




  0%|          | 1055/724004 [01:21<15:31:16, 12.94it/s]




  0%|          | 1057/724004 [01:22<19:43:48, 10.18it/s]




  0%|          | 1059/724004 [01:22<22:18:41,  9.00it/s]




  0%|          | 1061/724004 [01:22<26:35:57,  7.55it/s]




  0%|          | 1062/724004 [01:22<36:39:01,  5.48it/s]




  0%|          | 1063/724004 [01:23<36:24:59,  5.51it/s]




  0%|          | 1065/724004 [01:23<29:10:03,  6.88it/s]




  0%|          | 1069/724004 [01:23<24:24:50,  8.23it/s]




  0%|          | 1074/724004 [01:23<19:43:07, 10.18it/s]




  0%|          | 1076/724004 [01:24<22:53:24,  8.77it/s]




  0%|          | 1079/724004 [01:24<22:58:00,  8.74it/s]




  0%|          | 1081/724004 [01:24<26:14:01,  7.65it/s]




  0%|          | 1083/724004 [01:24<23:51:53,  8.41it/s]




  0%|          | 1086/724004 [01:25<18:44:24, 10.72it/s]




  0%|          | 1088/72

  0%|          | 1398/724004 [01:47<13:11:23, 15.22it/s]




  0%|          | 1401/724004 [01:47<19:54:22, 10.08it/s]




  0%|          | 1406/724004 [01:47<16:40:30, 12.04it/s]




  0%|          | 1411/724004 [01:48<13:18:36, 15.08it/s]




  0%|          | 1414/724004 [01:48<13:21:39, 15.02it/s]




  0%|          | 1419/724004 [01:48<11:01:46, 18.20it/s]




  0%|          | 1425/724004 [01:48<9:32:53, 21.02it/s] 




  0%|          | 1430/724004 [01:48<10:55:11, 18.38it/s]




  0%|          | 1433/724004 [01:49<19:23:22, 10.35it/s]




  0%|          | 1435/724004 [01:49<17:57:28, 11.18it/s]




  0%|          | 1438/724004 [01:49<15:06:05, 13.29it/s]




  0%|          | 1440/724004 [01:50<16:58:16, 11.83it/s]




  0%|          | 1443/724004 [01:50<13:55:10, 14.42it/s]




  0%|          | 1446/724004 [01:50<16:07:13, 12.45it/s]




  0%|          | 1448/724004 [01:50<21:47:08,  9.21it/s]




  0%|          | 1450/724004 [01:50<20:29:12,  9.80it/s]




  0%|          | 1454/72

...

  2%|▏         | 12355/724004 [16:55<14:59:36, 13.18it/s]




  2%|▏         | 12358/724004 [16:56<17:44:47, 11.14it/s]




  2%|▏         | 12362/724004 [16:56<18:10:05, 10.88it/s]




  2%|▏         | 12364/724004 [16:56<19:17:47, 10.24it/s]




  2%|▏         | 12367/724004 [16:57<19:51:56,  9.95it/s]




  2%|▏         | 12369/724004 [16:57<16:52:28, 11.71it/s]




  2%|▏         | 12372/724004 [16:57<14:28:06, 13.66it/s]




  2%|▏         | 12377/724004 [16:57<11:43:28, 16.86it/s]




  2%|▏         | 12380/724004 [16:57<15:34:30, 12.69it/s]




  2%|▏         | 12382/724004 [16:58<26:17:16,  7.52it/s]




  2%|▏         | 12384/724004 [16:58<34:22:17,  5.75it/s]




  2%|▏         | 12386/724004 [16:59<37:50:08,  5.22it/s]




  2%|▏         | 12391/724004 [16:59<28:55:38,  6.83it/s]




  2%|▏         | 12393/724004 [16:59<29:42:59,  6.65it/s]




  2%|▏         | 12395/724004 [17:00<32:58:13,  6.00it/s]




  2%|▏         | 12397/724004 [17:00<26:10:21,  7.55it/s]




  2%|▏  

  2%|▏         | 12645/724004 [17:21<26:38:01,  7.42it/s]




  2%|▏         | 12647/724004 [17:21<22:30:14,  8.78it/s]




  2%|▏         | 12654/724004 [17:21<17:03:23, 11.58it/s]




  2%|▏         | 12657/724004 [17:22<24:49:01,  7.96it/s]




  2%|▏         | 12659/724004 [17:22<23:38:22,  8.36it/s]




  2%|▏         | 12661/724004 [17:22<25:01:44,  7.89it/s]




  2%|▏         | 12663/724004 [17:22<20:42:43,  9.54it/s]




  2%|▏         | 12665/724004 [17:22<18:02:57, 10.95it/s]




  2%|▏         | 12667/724004 [17:23<22:10:17,  8.91it/s]




  2%|▏         | 12670/724004 [17:23<17:58:35, 10.99it/s]




  2%|▏         | 12674/724004 [17:23<14:41:18, 13.45it/s]




  2%|▏         | 12677/724004 [17:23<12:37:24, 15.65it/s]




  2%|▏         | 12681/724004 [17:23<11:17:20, 17.50it/s]




  2%|▏         | 12684/724004 [17:24<20:57:54,  9.42it/s]




  2%|▏         | 12687/724004 [17:24<20:44:32,  9.53it/s]




  2%|▏         | 12691/724004 [17:24<16:35:51, 11.90it/s]




  2%|▏  

  2%|▏         | 12963/724004 [17:46<17:17:46, 11.42it/s]




  2%|▏         | 12965/724004 [17:47<24:34:52,  8.04it/s]




  2%|▏         | 12968/724004 [17:47<19:23:33, 10.18it/s]




  2%|▏         | 12970/724004 [17:47<22:16:44,  8.87it/s]




  2%|▏         | 12972/724004 [17:47<20:46:07,  9.51it/s]




  2%|▏         | 12974/724004 [17:48<31:41:14,  6.23it/s]




  2%|▏         | 12976/724004 [17:48<27:26:11,  7.20it/s]




  2%|▏         | 12980/724004 [17:49<25:26:19,  7.76it/s]




  2%|▏         | 12982/724004 [17:49<24:46:45,  7.97it/s]




  2%|▏         | 12986/724004 [17:49<19:28:44, 10.14it/s]




  2%|▏         | 12989/724004 [17:49<18:55:30, 10.44it/s]




  2%|▏         | 12992/724004 [17:49<18:40:19, 10.58it/s]




  2%|▏         | 12994/724004 [17:50<23:24:34,  8.44it/s]




  2%|▏         | 12996/724004 [17:50<28:22:50,  6.96it/s]




  2%|▏         | 12999/724004 [17:50<24:20:29,  8.11it/s]




  2%|▏         | 13001/724004 [17:51<25:43:26,  7.68it/s]




  2%|▏  

  2%|▏         | 13234/724004 [18:13<18:22:16, 10.75it/s]




  2%|▏         | 13239/724004 [18:14<15:19:03, 12.89it/s]




  2%|▏         | 13242/724004 [18:14<12:43:26, 15.52it/s]




  2%|▏         | 13245/724004 [18:14<15:43:20, 12.56it/s]




  2%|▏         | 13251/724004 [18:14<12:19:43, 16.01it/s]




  2%|▏         | 13256/724004 [18:15<12:39:33, 15.60it/s]




  2%|▏         | 13259/724004 [18:15<12:25:02, 15.90it/s]




  2%|▏         | 13262/724004 [18:15<16:21:18, 12.07it/s]




  2%|▏         | 13264/724004 [18:15<14:59:43, 13.17it/s]




  2%|▏         | 13267/724004 [18:15<12:42:37, 15.53it/s]




  2%|▏         | 13269/724004 [18:16<15:13:59, 12.96it/s]




  2%|▏         | 13271/724004 [18:16<19:50:12,  9.95it/s]




  2%|▏         | 13276/724004 [18:16<17:20:55, 11.38it/s]




  2%|▏         | 13280/724004 [18:16<13:38:59, 14.46it/s]




  2%|▏         | 13283/724004 [18:16<13:27:47, 14.66it/s]




  2%|▏         | 13286/724004 [18:17<11:38:02, 16.97it/s]




  2%|▏  

  2%|▏         | 13539/724004 [18:40<24:31:17,  8.05it/s]




  2%|▏         | 13541/724004 [18:40<21:46:45,  9.06it/s]




  2%|▏         | 13543/724004 [18:41<24:27:49,  8.07it/s]




  2%|▏         | 13545/724004 [18:41<22:33:21,  8.75it/s]




  2%|▏         | 13548/724004 [18:41<18:12:03, 10.84it/s]




  2%|▏         | 13550/724004 [18:41<17:04:39, 11.56it/s]




  2%|▏         | 13552/724004 [18:41<14:58:31, 13.18it/s]




  2%|▏         | 13554/724004 [18:42<20:45:39,  9.51it/s]




  2%|▏         | 13556/724004 [18:42<18:10:02, 10.86it/s]




  2%|▏         | 13558/724004 [18:42<23:19:28,  8.46it/s]




  2%|▏         | 13563/724004 [18:42<17:43:36, 11.13it/s]




  2%|▏         | 13566/724004 [18:42<19:23:42, 10.17it/s]




  2%|▏         | 13568/724004 [18:43<20:53:22,  9.45it/s]




  2%|▏         | 13571/724004 [18:43<16:40:21, 11.84it/s]




  2%|▏         | 13575/724004 [18:43<15:04:52, 13.09it/s]




  2%|▏         | 13580/724004 [18:43<14:56:33, 13.21it/s]




  2%|▏  

In [309]:
to_save

[{'id': 52036250,
  'loc': 'Greensboro,NC',
  'profile_image_url': 'http://pbs.twimg.com/profile_images/1178198434995675137/PfvoJYs3_normal.jpg',
  'faces': None},
 {'id': 1691831382,
  'loc': 'Jacksonville,FL',
  'profile_image_url': 'http://pbs.twimg.com/profile_images/1229811434625916929/mc18iUC6_normal.jpg',
  'faces': [{'face_token': 'e2807b974ec3e12909b0589ad48d06c8',
    'face_rectangle': {'top': 158, 'left': 95, 'width': 210, 'height': 210},
    'attributes': {'gender': {'value': 'Male'}, 'age': {'value': 31}}}]},
 {'id': 358091193,
  'loc': 'Jackson,MI',
  'profile_image_url': 'http://pbs.twimg.com/profile_images/1233857782824263681/lhWwrTSo_normal.jpg',
  'faces': [{'face_token': '9d075870772c17bbbf7e8cc3d28ab923',
    'face_rectangle': {'top': 153, 'left': 194, 'width': 107, 'height': 107},
    'attributes': {'gender': {'value': 'Male'}, 'age': {'value': 66}}}]},
 {'id': 949656749635612673,
  'loc': 'Las Vegas,NV',
  'profile_image_url': 'http://pbs.twimg.com/profile_images/